In [1]:
!wget https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar -O tabular-dl-tabr.tar.gz
!tar -xvf tabular-dl-tabr.tar.gz

--2024-10-15 08:26:50--  https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar
Resolving huggingface.co (huggingface.co)... 13.35.210.114, 13.35.210.66, 13.35.210.77, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1b/25/1b25d6e2556c827abfaf6ba9da6021338691cafc39fe9d77986955ae0a69243d/0d74e4b96febb48fbe4dd2d851f8638b5738354f82c3183f92adbd2abf2f256b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.tar%3B+filename%3D%22data.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1729240011&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTI0MDAxMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xYi8yNS8xYjI1ZDZlMjU1NmM4MjdhYmZhZjZiYTlkYTYwMjEzMzg2OTFjYWZjMzlmZTlkNzc5ODY5NTVhZTBhNjkyNDNkLzBkNzRlNGI5NmZlYmI0OGZiZTRkZDJkODUxZjg2MzhiNTczODM1NGY4MmMzMTgzZjkyYWRiZDJhYmYyZjI1Nm

In [2]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.8 MB/s eta 0:00:00


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.8 MB/s eta 0:00:00


In [4]:
!pip install psutil

In [5]:
import pandas as pd
import numpy as np
import optuna
import torch.optim
import psutil

from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import QuantileTransformer

In [6]:
tnr = TabNetRegressor()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [7]:
X_train = np.load('/content/data/regression-num-medium-0-Ailerons/X_num_train.npy')
y_train = np.load('/content/data/regression-num-medium-0-Ailerons/Y_train.npy').reshape(-1, 1)

X_valid = np.load('/content/data/regression-num-medium-0-Ailerons/X_num_val.npy')
y_valid = np.load('/content/data/regression-num-medium-0-Ailerons/Y_val.npy').reshape(-1, 1)

X_test = np.load('/content/data/regression-num-medium-0-Ailerons/X_num_test.npy')
y_test = np.load('/content/data/regression-num-medium-0-Ailerons/Y_test.npy').reshape(-1, 1)

In [8]:
normalizer = QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X_train.shape[0] // 30, 1000), 10),
            subsample=1_000_000_000,
            )
normalizer.fit_transform(X_train)

array([[-1.6914173 ,  1.2977961 ,  0.6379428 , ...,  0.05503214,
         0.26635137, -0.24605119],
       [ 0.49931866,  0.7170322 ,  0.5998753 , ...,  0.05503214,
        -0.67695796, -1.0968035 ],
       [-0.31967288, -0.29906526,  1.2449065 , ...,  0.05503214,
         0.784747  ,  0.7375054 ],
       ...,
       [-0.9446696 ,  2.8350697 , -0.34041727, ...,  0.05503214,
         0.784747  ,  0.8282596 ],
       [ 0.7776394 ,  0.5262046 , -1.9667383 , ...,  0.05503214,
         1.5453691 ,  1.5855004 ],
       [ 0.82457423, -1.2887294 , -0.16178727, ...,  0.05503214,
         1.0685635 ,  0.90253806]], dtype=float32)

In [9]:
#Baseline
tnr.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['mae', 'rmse'],
)

epoch 0  | loss: 0.21079 | train_mae: 0.1153699979186058| train_rmse: 0.19348999857902527| valid_mae: 0.11743000149726868| valid_rmse: 0.27685999870300293|  0:00:02s
epoch 1  | loss: 0.0235  | train_mae: 0.028759999200701714| train_rmse: 0.059700001031160355| valid_mae: 0.028289999812841415| valid_rmse: 0.05443999916315079|  0:00:03s
epoch 2  | loss: 0.00597 | train_mae: 0.02199999988079071| train_rmse: 0.07370000332593918| valid_mae: 0.02047000080347061| valid_rmse: 0.0767500028014183|  0:00:03s
epoch 3  | loss: 0.00262 | train_mae: 0.011909999884665012| train_rmse: 0.0771000012755394| valid_mae: 0.009739999659359455| valid_rmse: 0.06751000136137009|  0:00:04s
epoch 4  | loss: 0.00166 | train_mae: 0.0031300000846385956| train_rmse: 0.007000000216066837| valid_mae: 0.0028099999763071537| valid_rmse: 0.0055599999614059925|  0:00:04s
epoch 5  | loss: 0.00124 | train_mae: 0.0017099999822676182| train_rmse: 0.0038499999791383743| valid_mae: 0.0014400000218302011| valid_rmse: 0.002709999913

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [10]:
def objective(trial):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'momentum': trial.suggest_float('momentum', 0.01, 0.4),
    }

    model = TabNetRegressor(**params, n_a=params['n_d'])
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], patience=10)

    # Evaluate model performance
    score = model.best_cost  # or any other metric

    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2024-10-15 08:29:59,835] A new study created in memory with name: no-name-5460e492-9117-4c18-b92d-a3cac3984662
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 7.09352 | val_0_mse: 6.501930236816406|  0:00:01s
epoch 1  | loss: 4.34854 | val_0_mse: 8.90542984008789|  0:00:03s
epoch 2  | loss: 9.11763 | val_0_mse: 14.728400230407715|  0:00:04s
epoch 3  | loss: 2.97799 | val_0_mse: 0.38155001401901245|  0:00:06s
epoch 4  | loss: 1.11328 | val_0_mse: 0.5538700222969055|  0:00:07s
epoch 5  | loss: 1.83796 | val_0_mse: 0.24327999353408813|  0:00:09s
epoch 6  | loss: 1.52063 | val_0_mse: 0.015189999714493752|  0:00:11s
epoch 7  | loss: 0.24386 | val_0_mse: 0.011189999990165234|  0:00:12s
epoch 8  | loss: 0.04717 | val_0_mse: 0.01671000011265278|  0:00:14s
epoch 9  | loss: 0.03192 | val_0_mse: 0.015320000238716602|  0:00:15s
epoch 10 | loss: 0.01232 | val_0_mse: 0.0013699999544769526|  0:00:17s
epoch 11 | loss: 0.01024 | val_0_mse: 0.0010000000474974513|  0:00:19s
epoch 12 | loss: 0.00531 | val_0_mse: 0.001069999998435378|  0:00:20s
epoch 13 | loss: 0.00271 | val_0_mse: 0.00023999999393709004|  0:00:21s
epoch 14 | loss: 0.00182 | val

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 08:31:42,692] Trial 0 finished with value: 4.379454367153812e-06 and parameters: {'n_d': 51, 'n_steps': 8, 'gamma': 1.4232258267650426, 'n_independent': 5, 'n_shared': 3, 'momentum': 0.22095787388539703}. Best is trial 0 with value: 4.379454367153812e-06.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 6.97577 | val_0_mse: 4.5311198234558105|  0:00:01s
epoch 1  | loss: 2.81298 | val_0_mse: 1.0567799806594849|  0:00:02s
epoch 2  | loss: 0.80108 | val_0_mse: 0.045049998909235|  0:00:03s
epoch 3  | loss: 0.68836 | val_0_mse: 0.20886999368667603|  0:00:04s
epoch 4  | loss: 0.73545 | val_0_mse: 0.17124000191688538|  0:00:05s
epoch 5  | loss: 0.29142 | val_0_mse: 0.0842600017786026|  0:00:06s
epoch 6  | loss: 0.18101 | val_0_mse: 0.07124999910593033|  0:00:07s
epoch 7  | loss: 0.57159 | val_0_mse: 1.0020999908447266|  0:00:08s
epoch 8  | loss: 0.13956 | val_0_mse: 0.06532999873161316|  0:00:09s
epoch 9  | loss: 0.01558 | val_0_mse: 0.006680000107735395|  0:00:10s
epoch 10 | loss: 0.0064  | val_0_mse: 0.0021100000012665987|  0:00:11s
epoch 11 | loss: 0.00548 | val_0_mse: 0.0010400000028312206|  0:00:12s
epoch 12 | loss: 0.00412 | val_0_mse: 0.0016700000269338489|  0:00:14s
epoch 13 | loss: 0.00324 | val_0_mse: 0.0012600000482052565|  0:00:15s
epoch 14 | loss: 0.00302 | val_

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 08:32:24,565] Trial 1 finished with value: 0.00011966858437517658 and parameters: {'n_d': 27, 'n_steps': 7, 'gamma': 1.3453073567903164, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.231788841117413}. Best is trial 0 with value: 4.379454367153812e-06.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 10:10:58,230] Trial 97 finished with value: 5.121512913319748e-06 and parameters: {'n_d': 34, 'n_steps': 6, 'gamma': 1.9574774408668043, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.2441657243503528}. Best is trial 20 with value: 1.2763081258526654e-07.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.40433 | val_0_mse: 4.383780002593994|  0:00:00s
epoch 1  | loss: 0.35986 | val_0_mse: 0.874459981918335|  0:00:01s
epoch 2  | loss: 0.13455 | val_0_mse: 0.298799991607666|  0:00:02s
epoch 3  | loss: 0.15018 | val_0_mse: 0.04992000013589859|  0:00:03s
epoch 4  | loss: 0.04524 | val_0_mse: 0.028769999742507935|  0:00:03s
epoch 5  | loss: 0.01629 | val_0_mse: 0.004509999882429838|  0:00:04s
epoch 6  | loss: 0.00379 | val_0_mse: 0.0007699999841861427|  0:00:05s
epoch 7  | loss: 0.00109 | val_0_mse: 0.00011000000085914508|  0:00:06s
epoch 8  | loss: 0.00056 | val_0_mse: 1.9999999494757503e-05|  0:00:07s
epoch 9  | loss: 0.00039 | val_0_mse: 0.0     |  0:00:08s
epoch 10 | loss: 0.00036 | val_0_mse: 0.0     |  0:00:08s
epoch 11 | loss: 0.00036 | val_0_mse: 0.0     |  0:00:09s
epoch 12 | loss: 0.00035 | val_0_mse: 0.0     |  0:00:10s
epoch 13 | loss: 0.00035 | val_0_mse: 0.0     |  0:00:11s
epoch 14 | loss: 0.00034 | val_0_mse: 0.0     |  0:00:11s
epoch 15 | loss: 0.00034 | 

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 10:11:19,945] Trial 98 finished with value: 3.44907903127023e-07 and parameters: {'n_d': 15, 'n_steps': 6, 'gamma': 1.7898865154198038, 'n_independent': 2, 'n_shared': 3, 'momentum': 0.2824477472202193}. Best is trial 20 with value: 1.2763081258526654e-07.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3.46404 | val_0_mse: 4.334219932556152|  0:00:01s
epoch 1  | loss: 3.55386 | val_0_mse: 5.557759761810303|  0:00:02s
epoch 2  | loss: 1.79668 | val_0_mse: 0.48489999771118164|  0:00:03s
epoch 3  | loss: 2.04503 | val_0_mse: 3.0330801010131836|  0:00:04s
epoch 4  | loss: 2.90141 | val_0_mse: 0.5649399757385254|  0:00:05s
epoch 5  | loss: 0.89841 | val_0_mse: 0.08320000022649765|  0:00:06s
epoch 6  | loss: 0.33335 | val_0_mse: 0.03303999826312065|  0:00:07s
epoch 7  | loss: 0.20255 | val_0_mse: 0.020800000056624413|  0:00:08s
epoch 8  | loss: 0.07586 | val_0_mse: 0.007420000154525042|  0:00:09s
epoch 9  | loss: 0.0186  | val_0_mse: 0.002689999993890524|  0:00:10s
epoch 10 | loss: 0.00572 | val_0_mse: 0.0003800000122282654|  0:00:11s
epoch 11 | loss: 0.00266 | val_0_mse: 0.000750000006519258|  0:00:12s
epoch 12 | loss: 0.00196 | val_0_mse: 0.0001900000061141327|  0:00:13s
epoch 13 | loss: 0.00147 | val_0_mse: 0.00018000000272877514|  0:00:14s
epoch 14 | loss: 0.001   | va

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-10-15 10:11:55,628] Trial 99 finished with value: 4.689306297223084e-05 and parameters: {'n_d': 47, 'n_steps': 7, 'gamma': 1.8600574189631538, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.3621830700916767}. Best is trial 20 with value: 1.2763081258526654e-07.


In [11]:
best_params

{'n_d': 35,
 'n_steps': 7,
 'gamma': 1.9187448447979616,
 'n_independent': 1,
 'n_shared': 3,
 'momentum': 0.2539679747829539}

In [12]:
from pytorch_tabnet.callbacks import Callback

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Memory Usage: {mem_info.rss / (1024 ** 2):.2f} MB")

class MemoryUsageCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print_memory_usage()

In [13]:
# Train final model using best hyperparameters
best_model = TabNetRegressor(**best_params,
                             n_a=best_params['n_d'],
                             optimizer_fn=torch.optim.Adam,
                             optimizer_params=dict(lr=2e-2)
                             )
best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['mae', 'rmse'],
    callbacks=[MemoryUsageCallback()]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 4.1191  | train_mae: 1.7066600322723389| train_rmse: 3.2469799518585205| valid_mae: 1.9961899518966675| valid_rmse: 3.8357200622558594|  0:00:01s
Memory Usage: 1231.29 MB
epoch 1  | loss: 0.72846 | train_mae: 0.9191399812698364| train_rmse: 1.2814500331878662| valid_mae: 0.8994899988174438| valid_rmse: 1.290779948234558|  0:00:02s
Memory Usage: 1231.29 MB
epoch 2  | loss: 0.53748 | train_mae: 0.7338600158691406| train_rmse: 1.276379942893982| valid_mae: 0.7178999781608582| valid_rmse: 1.297659993171692|  0:00:03s
Memory Usage: 1231.29 MB
epoch 3  | loss: 0.92815 | train_mae: 0.8736100196838379| train_rmse: 1.1634199619293213| valid_mae: 0.8591700196266174| valid_rmse: 1.1545100212097168|  0:00:05s
Memory Usage: 1231.29 MB
epoch 4  | loss: 0.60379 | train_mae: 0.17367999255657196| train_rmse: 0.29613998532295227| valid_mae: 0.162650004029274| valid_rmse: 0.2818099856376648|  0:00:06s
Memory Usage: 1231.29 MB
epoch 5  | loss: 0.20638 | train_mae: 0.5057799816131592| trai

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report

preds = best_model.predict(X_test)
y_true = y_test

mae_test = mean_absolute_error(y_pred=preds, y_true=y_true)
mse = mean_squared_error(y_pred=preds, y_true=y_true)
rmse_test = np.sqrt(mean_squared_error(y_true, y_pred=preds))
r2_test = r2_score(y_true=y_true, y_pred=preds)

print("Best Valid RMSE:", best_model.best_cost)
print("Test MAE:", mae_test)
print("Test RMSE:", rmse_test)
print("R-squared:", r2_test)

Best Valid RMSE: 0.00041069344
Test MAE: 0.0003236484
Test RMSE: 0.00042569902
R-squared: -0.12583589553833008


In [ ]:
base_preds = tnr.predict(X_test)

base_mae_test = mean_absolute_error(y_pred=base_preds, y_true=y_true)
base_mse = mean_squared_error(y_pred=base_preds, y_true=y_true)
base_rmse_test = np.sqrt(mean_squared_error(y_true, y_pred=base_preds))

print("Best Valid Score:", tnr.best_cost)
print("Test MAE:", base_mae_test)
print("Test RMSE:", base_rmse_test)

Best Valid Score: 0.0007780627
Test MAE: 0.00063304434
Test RMSE: 0.00079291884
